# Goals 

The goal of this part of the tutorial is build your skills in: 
   - Asking good questions 
   - Working with data  
   - Learn about a few types of regression
       - Linear regression
       - Logistic regression 
       - Lasso regression 
   - Fit a model 
   - Evaluate the model, using accuracy and ROC 
   - Predicting whether a movie will win an Oscar

# Ask a good question 

We want to know what will make a movie an Oscar Winner, but how can we ask this in a way that we can answer it given the data that we have? 

*your answer here*

# Regression 

## Linear Regression

Error minimization: Ordinary least squares 
Outcome: Continuous 

## Logistic Regression 

Error minimization: Maximum likelihood 
Outcome: Binary 

## Lasso Regression 

# Fit models 

# Evaluate model performance 

## Look at accuracy 

One way to evaluate our model is look at its accuracy. For example, if we are classifying movies as Oscar winners or not. What proportion of movies can we correctly identify as Oscar winners? 

## Error Rates 

Confusion matrices are great because they allow us to see what types of errors are model makes! If our model makes systematic errors, we need to know that! It is easiest to think about error rates in the context of a binary confusion matrix. 

<img src="img/binaryconfusionmatrix.png"
     alt="Example Great Movie"
     style="float: left; margin-right: 100px;"/>

Putting this in terms of our example: 
   - True Positive = You predicted a movie would get an Oscar, and it did! 
   - True Negative = You predicted a movie would *not* get an Oscar, and it did not get one. 
   - False Positive = You predicted a movie would get an Oscar, and it did *not*.
   - False Negative = You predicted a movie would *not* get an Oscar, but it did. 

## Explore your errors

Exploring the types of errors made by your model is super important! I recommend doing at least a simple check whenever you check model accuracy. 

While the internet and our newfound scraping skills can give us a ton of data, to develop good models, we need to consider that data with some skepticism and look at the errors our models make. 

For example, movies makers are notorious for overrepresenting some parts of society while underrepresenting others. While a movie audience is made of 50% women, only 24% of movie leads are women. 68% of all female characters were white in the top 100 films of 2017. 16% were Black, 7% were Asian, 7% were Latina, and 2% were of another race or ethnicity [[Women & Hollywood](https://womenandhollywood.com/resources/statistics/)]. 

This means that if we want to use movie data from previous, we have to be aware that most of the leads and speaking characters are going to have light skin and ~76% of them will be male.

One clear example of how this can go wrong can be demonstrated by facial recognition software. Joy was a graduate student at the MIT Media Lab, working facial detection software, but she had trouble testing her code because the out of the box face detection software never detected her face. To test her code, she had to wear a white mask to be recognized. 

<img src="https://i.ytimg.com/vi/162VzSzzoPs/maxresdefault.jpg"
     alt="Bias in ML"
     style="float: left; margin-right: 100px;"/>
     
Because the training data contained many light-skinned faces (similar to those below), the software was incapable of recognizing faces that deviated from that norm. 

<img src="https://productofsocietydotorg.files.wordpress.com/2016/02/faces-of-american-power-are-mostly-white-1456497978018-facebookjumbo.png?w=1024"
     alt="Bias in ML"
     style="float: left; margin-right: 100px;"/>


In [ ]:

## ROC curves 